# Multi-World Simulation and Domain Randomization

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-35QaaoiXx6VDmVNBOEtmpLs9VBs)

This notebook demonstrates Newton's **Selection API** and **multi-world simulation** for domain randomization in reinforcement learning. We'll simulate multiple robots in parallel with randomized material properties.

Key topics:
- Creating multiple simulation worlds efficiently
- Using `ArticulationView` to batch-modify properties
- Randomizing friction coefficients for domain randomization
- Notifying solvers of runtime property changes


## Setup and Imports


In [ ]:
import newton
import newton.examples
import warp as wp
import numpy as np
from newton.selection import ArticulationView
from newton.solvers import SolverNotifyFlags
from tqdm.notebook import trange


## Why Multi-World Simulation?

**Domain Randomization** is crucial for training robust RL policies:
- Randomize physics parameters (friction, mass, damping)
- Train on diverse conditions → better sim-to-real transfer
- Parallel simulation → faster training

**Newton's Multi-World Architecture:**
- Single `Model` contains multiple independent worlds
- Batch operations via `ArticulationView`
- GPU-accelerated parallel simulation
- Efficient memory layout for vectorized operations


## Create Multi-World Scene

We'll create 16 parallel worlds, each containing a quadruped robot.


In [ ]:
# Number of parallel worlds
num_worlds = 16

# Create a template for a single world
world_template = newton.ModelBuilder()
world_template.add_mjcf(
    newton.examples.get_asset("nv_ant.xml"),
    ignore_names=["floor", "ground"],  # We'll add our own ground
    collapse_fixed_joints=False,
)

print(f"Template world: {world_template.body_count} bodies, {world_template.joint_count} joints")


In [ ]:
# Create the main scene and replicate the template
scene = newton.ModelBuilder()
scene.add_ground_plane()  # Shared ground across all worlds
scene.replicate(world_template, num_worlds=num_worlds)

# Finalize the model
model = scene.finalize()

print(f"\nFinal model with {num_worlds} worlds:")
print(f"  Total bodies: {model.body_count}")
print(f"  Total joints: {model.joint_count}")
print(f"  Total shapes: {model.shape_count}")


## Create ArticulationView

The `ArticulationView` provides batch access to articulations across worlds.


In [ ]:
# Create ArticulationView for all ant robots
# This gives us batch access to all ants across all worlds
ants = ArticulationView(
    model,
    "ant",  # Articulation name from MJCF
    verbose=True,
    exclude_joint_types=[newton.JointType.FREE],  # Exclude floating base
)

print(f"\nArticulationView created:")
print(f"  Number of articulations: {ants.count}")
print(f"  DOFs per articulation: {ants.joint_dof_count}")
print(f"  Shapes per articulation: {ants.shape_count}")
print(f"  Bodies per articulation: {ants.link_count}")


## Setup Solver and States


In [ ]:
# Create solver
solver = newton.solvers.SolverMuJoCo(model, njmax=50, nconmax=250)

# Create states
state_0 = model.state()
state_1 = model.state()
control = model.control()

# Evaluate initial FK
newton.eval_fk(model, model.joint_q, model.joint_qd, state_0)

print("Solver and states initialized")


## Material Randomization Kernel

Define a Warp kernel to randomize friction coefficients.


In [ ]:
@wp.kernel
def randomize_friction_kernel(
    mu: wp.array2d(dtype=float),  # [num_worlds, num_shapes]
    seed: int,
    shape_count: int,
    randomize_per_world: bool,
):
    """Randomize friction coefficients.
    
    Args:
        mu: Friction coefficient array to modify
        seed: Random seed
        shape_count: Number of shapes per world
        randomize_per_world: If True, all shapes in a world get same friction
    """
    i, j = wp.tid()
    
    if randomize_per_world:
        # Same friction for all shapes in this world
        rng = wp.rand_init(seed, i)
    else:
        # Different friction for each shape
        rng = wp.rand_init(seed, i * shape_count + j)
    
    # Random friction between 0.0 and 1.0
    mu[i, j] = wp.randf(rng)


print("Friction randomization kernel defined")


## Store Default States

Save default configurations for resetting.


In [ ]:
# Store default root transforms and velocities
default_root_transforms = wp.clone(ants.get_root_transforms(model))
default_root_velocities = wp.clone(ants.get_root_velocities(model))

# Set DOFs to middle of their range
dof_limit_lower = ants.get_attribute("joint_limit_lower", model)
dof_limit_upper = ants.get_attribute("joint_limit_upper", model)
default_dof_positions = wp.empty_like(dof_limit_lower)

# Compute middle values
@wp.kernel
def compute_middle_kernel(
    lower: wp.array2d(dtype=float),
    upper: wp.array2d(dtype=float),
    middle: wp.array2d(dtype=float),
):
    i, j = wp.tid()
    middle[i, j] = 0.5 * (lower[i, j] + upper[i, j])

wp.launch(
    compute_middle_kernel,
    dim=default_dof_positions.shape,
    inputs=[dof_limit_lower, dof_limit_upper, default_dof_positions],
)

default_dof_velocities = wp.clone(ants.get_dof_velocities(model))

print("Default states stored")


## Reset Function with Material Randomization

This function resets the simulation and randomizes material properties.


In [ ]:
reset_count = 0

def reset_simulation(randomize_per_world=True):
    """Reset simulation with randomized materials."""
    global reset_count
    
    # Alternate forward/backward velocities
    if reset_count % 2 == 0:
        default_root_velocities.fill_(wp.spatial_vector(0.0, 5.0, 0.0, 0.0, 0.0, 0.0))
    else:
        default_root_velocities.fill_(wp.spatial_vector(0.0, -5.0, 0.0, 0.0, 0.0, 0.0))
    
    # Randomize friction coefficients
    material_mu = ants.get_attribute("shape_material_mu", model)
    wp.launch(
        randomize_friction_kernel,
        dim=material_mu.shape,
        inputs=[material_mu, reset_count, ants.shape_count, randomize_per_world],
    )
    
    # Apply randomized materials to model
    ants.set_attribute("shape_material_mu", model, material_mu)
    
    # IMPORTANT: Notify solver that material properties changed
    solver.notify_model_changed(SolverNotifyFlags.SHAPE_PROPERTIES)
    
    # Reset robot states
    ants.set_root_transforms(state_0, default_root_transforms)
    ants.set_root_velocities(state_0, default_root_velocities)
    ants.set_dof_positions(state_0, default_dof_positions)
    ants.set_dof_velocities(state_0, default_dof_velocities)
    
    reset_count += 1
    
    # Print friction range for first world
    mu_np = material_mu.numpy()
    print(f"Reset {reset_count}: Friction range [{mu_np.min():.3f}, {mu_np.max():.3f}]")


# Perform initial reset
reset_simulation(randomize_per_world=True)
print("\nInitial reset complete")


## CUDA Graph Capture

Capture the simulation loop as a CUDA graph for maximum performance.


In [ ]:
# Simulation parameters
fps = 60
frame_dt = 1.0 / fps
sim_substeps = 10
sim_dt = frame_dt / sim_substeps

# Create temporary state for swapping (needed for CUDA graph)
state_temp = model.state()

def simulate_step():
    global state_0, state_1
    """Run physics substeps with proper state swapping for CUDA graphs."""
    for i in range(sim_substeps):
        state_0.clear_forces()
        solver.step(state_0, state_1, control, None, sim_dt)
        state_0, state_1 = state_1, state_0

# Capture CUDA graph
graph = None
if wp.get_device().is_cuda and wp.is_mempool_enabled(wp.get_device()):
    print("Capturing CUDA graph for optimized execution...")
    with wp.ScopedCapture() as capture:
        simulate_step()
    graph = capture.graph
    print("CUDA graph captured successfully")
else:
    print("Running on CPU (no CUDA graph)")

print(f"\nSimulation: {fps} Hz, Substeps: {sim_substeps}")


## Simulation Loop

Run simulation with periodic resets and material randomization.


In [ ]:
# Create viewer with world offsets for visualization
viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)
viewer.set_world_offsets((4.0, 4.0, 0.0))  # Space out worlds in visualization

# Main simulation loop
num_frames = 300
sim_time = 0.0
reset_interval = 2.0  # Reset every 2 seconds
next_reset = reset_interval

for frame in trange(num_frames, desc="Simulating multi-world"):
    # Check if it's time to reset
    if sim_time >= next_reset:
        reset_simulation(randomize_per_world=True)
        next_reset = sim_time + reset_interval
    
    # Run physics substeps (use CUDA graph if available)
    if graph:
        wp.capture_launch(graph)
    else:
        simulate_step()
    
    # Log to viewer
    viewer.begin_frame(sim_time)
    viewer.log_state(state_0)
    viewer.end_frame()
    
    sim_time += frame_dt

print(f"\nSimulation complete! Total time: {sim_time:.2f} seconds")
print(f"Total resets: {reset_count}")
viewer


## Inspect Material Properties

Let's examine the randomized friction coefficients across worlds.


In [ ]:
# Get current friction coefficients
material_mu = ants.get_attribute("shape_material_mu", model)
mu_np = material_mu.numpy()

print("Friction Coefficient Statistics:")
print(f"  Shape: {mu_np.shape} (worlds × shapes)")
print(f"  Mean: {mu_np.mean():.3f}")
print(f"  Std: {mu_np.std():.3f}")
print(f"  Min: {mu_np.min():.3f}")
print(f"  Max: {mu_np.max():.3f}")

# Show friction for first 3 worlds
print("\nFriction per world (first 3 worlds):")
for i in range(min(3, num_worlds)):
    print(f"  World {i}: {mu_np[i, :5]} ... (showing first 5 shapes)")


## Other Randomizable Properties

The Selection API allows modifying many properties for domain randomization:


In [ ]:
# Example: Get other modifiable properties
print("Available shape properties:")
print("  - shape_material_mu (friction coefficient)")
print("  - shape_material_ke (contact stiffness)")
print("  - shape_material_kd (contact damping)")
print("  - shape_material_kf (friction stiffness)")

print("\nAvailable body properties:")
print("  - body_mass")
print("  - body_inertia")
print("  - body_com (center of mass)")

print("\nAvailable joint properties:")
print("  - joint_limit_lower")
print("  - joint_limit_upper")
print("  - joint_target_ke (PD stiffness)")
print("  - joint_target_kd (PD damping)")
print("  - joint_armature")

# Example: Randomize contact stiffness
contact_ke = ants.get_attribute("shape_material_ke", model)
print(f"\nCurrent contact stiffness shape: {contact_ke.shape}")
print(f"Contact stiffness range: [{contact_ke.numpy().min():.1e}, {contact_ke.numpy().max():.1e}]")


## Summary

In this notebook, we demonstrated:

1. **Multi-world creation** using `builder.replicate()`
2. **ArticulationView** for batch access to articulations
3. **Material randomization** with Warp kernels
4. **Runtime property modification** via Selection API
5. **Solver notification** with `notify_model_changed()`

### Key Concepts

**Multi-World Architecture:**
```python
scene.replicate(world_template, num_worlds=N)
```
Creates N independent copies efficiently with shared memory layout.

**ArticulationView:**
```python
view = ArticulationView(model, "robot_name")
view.get_attribute("property_name", model)  # Get [N_worlds, N_items]
view.set_attribute("property_name", model, values)  # Set batch
```

**Domain Randomization Pattern:**
1. Get property array: `prop = view.get_attribute(...)`
2. Randomize with kernel: `wp.launch(randomize_kernel, ...)`
3. Set back to model: `view.set_attribute(..., prop)`
4. Notify solver: `solver.notify_model_changed(flags)`

**Solver Notification Flags:**
- `SHAPE_PROPERTIES`: Material properties changed (friction, stiffness)
- `BODY_PROPERTIES`: Mass, inertia, COM changed
- `JOINT_PROPERTIES`: Joint limits, gains changed

### Benefits for RL

- **Parallel training**: 16+ environments in single simulation
- **Domain randomization**: Robust policies via diverse physics
- **Efficient**: GPU-accelerated batch operations
- **Flexible**: Randomize any physical property

### Next Steps

- Randomize other properties (mass, damping, joint limits)
- Implement curriculum learning (gradually increase randomization)
- Use with RL frameworks (Isaac Lab, rl_games)
- Add visual randomization (lighting, textures)
